In [3]:
import matplotlib
import pathlib
import os.path
from os import path
import mne
import mne_bids #brain imaging data structure #convert dataset to bits compatible dataset, .fif is actual data, will be able to run tools that expect bits data
from bids_validator import BIDSValidator
import numpy as np

In [4]:
matplotlib.use('Qt5Agg')  #interactive

In [5]:
data_path="/Users/Asmita_Hajra/Downloads/bidsexport" #path of BIDS data

## Structure of directory

In [6]:
mne_bids.print_dir_tree(data_path)

|bidsexport/
|--- .DS_Store
|--- CHANGES
|--- README
|--- dataset_description.json
|--- epochs_epo.fif
|--- evokeds_ave.fif
|--- participants.json
|--- participants.tsv
|--- task-meditation_events.json
|--- code/
|------ run_mw_experiment6.m
|--- stimuli/
|------ cancel.wav
|------ depth.wav
|------ expe_over.wav
|------ mind_wandering.wav
|------ rate_meditation.wav
|------ rate_mw.wav
|------ rate_tired.wav
|------ resume.wav
|------ resumed.wav
|------ resumemed.wav
|------ self.wav
|------ starting.wav
|------ time.wav
|------ valence.wav
|--- sub-001/
|------ .DS_Store
|------ ses-01/
|--------- .DS_Store
|--------- eeg/
|------------ sub-001_ses-01_task-meditation_channels.tsv
|------------ sub-001_ses-01_task-meditation_eeg.bdf
|------------ sub-001_ses-01_task-meditation_eeg.json
|------------ sub-001_ses-01_task-meditation_events.tsv
|------------ .ipynb_checkpoints/
|--------------- sub-001_ses-01_task-meditation_channels-checkpoint.tsv
|--------------- sub-001_ses-01_task-me

## Report

In [7]:
print(mne_bids.make_report(data_path)) #path to bids directory #doubt, no authors thus error

KeyError: 'Authors'

## Reading BIDS data-> analyzing raw data



In [12]:
# currently for finding X of dataset
def load_and_epoch_data(n): # n= no. of subjects 
     # INPUT: NUMBER OF SUBJECTS DATASET TO BE LOADED
     # OUTPUT: list of 2d array (length = no. of subjects * 2)
    raw_data_np=[]
    raw_data=[]
    subject_id=''
    session_id=''
    for i in range(n): #0-23
        for j in range(3): # no. of sessions
            # i is the subject number
            # j is the session number
            if(i<9):
                subject_id="00"+str(i+1)
            else:
                subject_id="0"+str(i+1)
                
            session_id="0"+str(j+1)
            
            # check if path exists 
            # print("hello"+ str(i)+""+str(j))
            dataset_path_to_validate=data_path+"/sub-"+subject_id+"/ses-"+session_id
            if(path.isdir(dataset_path_to_validate)): #validating if these sessions exist
                list_of_files = os.listdir(dataset_path_to_validate+"/eeg") # Checking if the folder is empty
                if len(list_of_files) != 0:
                    print("hey")
                    bids_path = mne_bids.BIDSPath(subject=subject_id, 
                              session=session_id,
                              task='meditation',
                              # run='01', #doubt what is run number
                              datatype='eeg', #will have a folder with datatype as name
                              root=data_path)
#                 if(BIDSValidator().is_bids(dataset_path_to_validate+"/eeg")):
#                     print("hellp")
                    raw = mne_bids.read_raw_bids(bids_path)
                    # print(raw.shape)'RawEDF', object has no attribute 'shape'
                    data, times = raw[:] # converting to ndarray
                    print (data.shape)
                    raw_data_np.append(data)
                    raw_data.append(raw) # list of raw data

    return raw_data, raw_data_np

no_of_subjects= 24
raw_data, raw_data_np = load_and_epoch_data(no_of_subjects)

hey
Extracting EDF parameters from /Users/Asmita_Hajra/Downloads/bidsexport/sub-001/ses-01/eeg/sub-001_ses-01_task-meditation_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /Users/Asmita_Hajra/Downloads/bidsexport/sub-001/ses-01/eeg/sub-001_ses-01_task-meditation_events.tsv.
The event "response" refers to multiple event values. Creating hierarchical event names.
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/4
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/4
    Renaming event: response -> response/4
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renam

In [13]:
len(raw_data_np)

40

In [14]:
type(raw_data_np)

list

In [77]:
# Saving the ndarray
arr = np.empty(40, object) # 40 objects
arr[:] = raw_data_np
np.save('/Users/Asmita_Hajra/mne-research/stored_data.npy', arr) # save

In [80]:
# Loading the nd array
loaded_raw_data = np.load('/Users/Asmita_Hajra/mne-research/stored_data.npy', allow_pickle=True) # load


In [18]:
for i in range(40):
    events=mne.find_events(raw_data[i]) 
    
#Observation-> Max no of events-> 2, 4, 8, 128, 254

<class 'mne.io.edf.edf.RawEDF'>
<class 'mne.io.edf.edf.RawEDF'>
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
87 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
87 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
101 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
107 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this ev


## About data
#### Stimulus is 128-> Onset of first Question
#### Response -> Response to question 1, 2 or 3
#### doubt what is STATUS-> 254
#### "2"-> Response 1 (this may be a response to question 1, 2 or 3)"
#### "4"-> Response 2 (this may be a response to question 1, 2 or 3)"
#### "8"-> Response 3 (this may be a response to question 1, 2 or 3)"
#### "16"-> involuntary response
#### "128"-> First question onset (most important marker)

## Finding info and visualizing for one subject's session

In [32]:
print(raw_data[3].info) # 68 EEG channels

<Info | 9 non-empty values
 bads: []
 ch_names: A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, A14, ...
 chs: 68 EEG, 10 MISC, 1 RESP, 1 STIM
 custom_ref_applied: False
 highpass: 0.0 Hz
 line_freq: 50
 lowpass: 52.0 Hz
 meas_date: 2014-04-12 16:57:33 UTC
 nchan: 80
 projs: []
 sfreq: 256.0 Hz
 subject_info: 4 items (dict)
>


In [60]:
print(type(raw_data[0]))
# raw_data[0].plot()
events = mne.find_events(raw_data[0]) # 3 events, [  2   4 128]

# raw_data[5].plot()
events = mne.find_events(raw_data[5]) # 4 events, [  2   4   8 128]

raw_data[21].plot()
events = mne.find_events(raw_data[21]) # 5 events, [  2   4   8 128 254]

# doubt difference in response/2 vs response/4 vs response/8
print("no of event 8", len(events[events[:, 2] == 8])) #all rows with event no 8
print(raw_data[21].info)
print("measuring date", raw_data[21].info['meas_date'])
print("sampling frequency",raw_data[21].info['sfreq']) #number of samples/second-> 256
print("line frquency", raw_data[21].info['line_freq']) #line frequency
print("subject information", raw_data[21].info['subject_info'])
print("events are now annotations", raw_data[21].annotations)
print("bads", raw_data[21].info['bads']) #none
print("10 channel names", raw_data[21].ch_names[:10])
print("info of a channel", raw_data[21].info['chs'][0])

<class 'mne.io.edf.edf.RawEDF'>
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
87 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
79 events found
Event IDs: [  2   4   8 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
113 events found
Event IDs: [  2   4   8 128 254]
no of event 8 20
<Info | 11 non-empty values
 bads: 1 items (A3)
 ch_names: A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, A14, ...
 chs: 68 EEG, 10 MISC, 1 RESP, 1 STIM
 custom_ref_applied: False
 dig: 67 items (3 Cardinal, 64 EEG)
 highpass: 0.0 Hz
 line_freq: 50
 lowpass: 52.0 Hz
 meas_date: 2014-04-17 08:03:35 UTC
 ncha

In [61]:
# Montages contain sensor positions in 3D (x, y, z in meters)

biosemi_montage = mne.channels.make_standard_montage('biosemi128')
biosemi_montage.plot(show_names=False)

raw_data[21].set_montage(biosemi_montage, on_missing="ignore")
raw_data[21].plot_sensors(ch_type='eeg')


Creating RawArray with float64 data, n_channels=128, n_times=1
    Range : 0 ... 0 =      0.000 ...     0.000 secs
Ready.


<Figure size 1280x1280 with 1 Axes>

In [ ]:
# lets assert a few things to find distribution of data
# sfreq, 

# biosemi_montage.plot(show_names=False, sphere=0.07)
# biosemi_montage.plot(show_names=False, sphere=(0.03, 0.02, 0.01, 0.075))

print(type(loaded_raw_data)) 
print(loaded_raw_data.shape) # 40
print(loaded_raw_data[0].shape) # (80, 696576)
for i in range(40): #0-23
        assert(loaded_raw_data[i].shape[0]==80) # checking if all have equal channels

In [133]:
# events = mne.find_events(raw) # doubt meaning of below
# doubt do we use events_True

#If True (default False), an event is created if the stim channel has a value different from 0 as its first sample. This is useful if an event at t=0s is present

## Creating evoked data